# Limpieza de Datos

La base de datos que limpiaremos contiene detalles de las transacciones de los usuarios, incluidos sus datos demográficos y su comportamiento de compra. Esta base incluye variables como el ID de usuario, edad, sexo, país, monto de compra, fecha de compra y categoría de producto. A través de su limpieza, buscaremos organizar y depurar la información para facilitar el análisis de las tendencias de consumo, las influencias demográficas en las decisiones de compra y la segmentación del mercado, lo que permitirá obtener insights valiosos sobre el comportamiento de los consumidores.

Los pasos de limpieza incluyen:
- Valores faltantes por columnas (Nulos)
- Renombrar las variables
- Conversión de datos
- Valores atípicos/inconsistentes en las variables numéricas
- Revisión de valores duplicados


In [26]:
# Bibliotecas necesarias
import pandas as pd


In [27]:
# Cargar los datos
data = 'compras_online.csv'
df = pd.read_csv(data)


## Valores Faltantes por columna (Nulos)

Eliminar estos valores es esencial para garantizar la precisión del análisis de datos, ya que pueden distorsionar los resultados, afectando la calidad de las predicciones y conclusiones. 

In [28]:
vls_nulos = df.isnull().sum()
print("Valores nulos por columna:")
print(vls_nulos)


Valores nulos por columna:
User ID                0
Age                 1500
Gender               750
Country             1200
Purchase Amount     1800
Purchase Date       1050
Product Category     900
dtype: int64


In [29]:
# Eliminar valores nulos

df_sin_nulos = df.dropna()
df_sin_nulos.isnull().sum()

User ID             0
Age                 0
Gender              0
Country             0
Purchase Amount     0
Purchase Date       0
Product Category    0
dtype: int64

Finalmente, podemos observar, que el número de observaciones se redujo notablemente al eliminar los valores nulos.

In [30]:
print("Número de filas antes de la eliminación:", len(df))  # df_original es tu DataFrame original
print("Número de filas después de la eliminación:", len(df_sin_nulos))

Número de filas antes de la eliminación: 15000
Número de filas después de la eliminación: 9076


## Renombrar las variables

Esto nos facilita el análisis, asegurando que sean claros los nombres de las variables a utilizar. También mejora la legibilidad del código, ayuda a evitar confusión durante el análisis y puede optimizar la interpretación de los resultados.

In [31]:
# Renombrar las variables

df = df_sin_nulos.rename(columns={
    'User ID': 'Usuario',
    'Age': 'Edad',
    'Gender': 'Género',
    'Country': 'Pais',
    'Purchase Amount': 'Monto_total',
    'Purchase Date': 'Fecha_compra',
    'Product Category': 'Categoria_producto'})

df

,Usuario,Edad,Género,Pais,Monto_total,Fecha_compra,Categoria_producto
0,1,56.0,Female,USA,331.79,2021-11-21,Sports
1,2,69.0,Male,Australia,335.72,2022-03-05,Home & Kitchen
3,4,32.0,Male,Germany,80.97,2023-06-08,Sports
6,7,38.0,Female,Canada,222.20,2022-02-23,Beauty
7,8,56.0,Male,USA,217.27,2021-09-08,Sports
...,...,...,...,...,...,...,...
14993,14994,54.0,Male,Australia,200.24,2021-11-26,Clothing
14995,14996,21.0,Male,UK,427.07,2023-08-20,Clothing
14996,14997,53.0,Other,UK,27.73,2021-07-23,Home & Kitchen
14997,14998,65.0,Female,Canada,108.57,2022-02-21,Beauty


## Conversión de Datos

Convertimos las variables Género y Categoría del producto de tipo texto a tipo categórico. Esto nos ayuda a optimizar el almacenamiento y mejorar nuestro análisis.

Inicialmente, vemos que las variables Género y Categoría del Producto, están cómo tipo texto, pero se podrían considerar categóricas.

In [32]:
dtype_original = df.dtypes
print(df.dtypes)

Usuario                 int64
Edad                  float64
Género                 object
Pais                   object
Monto_total           float64
Fecha_compra           object
Categoria_producto     object
dtype: object


Convertimos estas variables de float a category

In [33]:
df['Genero'] = df['Género'].astype('category')
df['Categoria_producto'] = df['Categoria_producto'].astype('category')

print(df.dtypes)

Usuario                  int64
Edad                   float64
Género                  object
Pais                    object
Monto_total            float64
Fecha_compra            object
Categoria_producto    category
Genero                category
dtype: object


## Valores atípicos/inconsistentes en las variables numéricas

In [ ]:
df_numerico = df_sin_nulos.select_dtypes(include=['float64', 'int64'])

Q1 = df_numerico.quantile(0.25)
Q3 = df_numerico.quantile(0.75)
IQR = Q3 - Q1
df_numerico = df_numerico[~((df_numerico < (Q1 - 1.5 * IQR)) | (df_numerico > (Q3 + 1.5 * IQR))).any(axis=1)]

print(f"Filas antes de eliminar valores atípicos: {df.shape[0]}")
print(f"Filas después de eliminar valores atípicos: {df_numerico.shape[0]}")
print("DataFrame después de eliminar valores atípicos:")
print(df_numerico)

Filas antes de eliminar valores atípicos: 9076
Filas después de eliminar valores atípicos: 9076
DataFrame después de eliminar valores atípicos:
       User ID   Age  Purchase Amount
0            1  56.0           331.79
1            2  69.0           335.72
3            4  32.0            80.97
6            7  38.0           222.20
7            8  56.0           217.27
...        ...   ...              ...
14993    14994  54.0           200.24
14995    14996  21.0           427.07
14996    14997  53.0            27.73
14997    14998  65.0           108.57
14998    14999  34.0           105.57

[9076 rows x 3 columns]


Podemos observar que ya en la base de datos no se encuentran valores atípicos o inconsistentes.

## Revisión valores duplicados

In [35]:
df_antes = df.copy()

df.drop_duplicates(inplace=True)

print(f"Número de filas con duplicados: {df_antes.shape[0]}")
print(f"Número de filas sin duplicados: {df.shape[0]}")

# Comprobar si hay duplicados restantes
duplicated = df.duplicated().sum()
print(f"Duplicados restantes: {duplicated}")


Número de filas con duplicados: 9076
Número de filas sin duplicados: 9076
Duplicados restantes: 0
